## 単語の分散表現

色をRGBとして表現すると、ベクトルとしてのコンパクトな表現、似た色かどうかの定量的な測定ができるようになる。

単語の意味についても同じように行えないか？
この問いに答えるのが単語の分散表現。

単語の分散表現は、単語を固定長ベクトルで表現する。

## 分布仮説
では、単語の意味をどのようにとらえるかだが、現在いろいろな方法が試されている中、多くの手法の中である共通のシンプルなアイデアに基づいていることがわかる。それが分布仮説。

分布仮説とは、単語の意味は周囲の単語によって形成されるというもの

つまり、単語自体に意味はなく、その単語のコンテキスト(文脈)によって意味が形成される。

以下、コンテキストはより機械学習よりの意味として「注目している単語の周囲に存在する単語」を指すとする。

どれくらい周囲まで見るのか(サイズ)はウィンドウサイズと呼ばれる。１であれば、単語の左右１つずつの単語のことを指す。

## カウントベース手法(統計的手法)

単語をベクトル化する最も単純な方法はカウントベース手法と呼ばれ、単語の周囲にどのような単語がどれくらい存在するのか数を数える手法である。

たとえば、"you say goodbye and i say hello."という文章があるとき、youについてみていくと、

youの一つとなりにはsayが１つある。sayのとなりには、you, goodby, i, helloが1つずつある。youの隣にはyou, iなどが0個となる。

これらをそれぞれの単語について集計し、行列にする。これを共起行列という。

|-|you|say|goodbye|and|i|say|hello|.|
|:--:|:--:|:--:|:--:|:--:|:--:|:--:|:--:|:--:|:--:|
|you|0|1|0|0|0|0|0|
|say|1|0|1|0|1|1|0|
|goodbye|
|and|
|i|
|say|
|hello|
|.|

これらの１行が単語それぞれのベクトルになる。

それでは、共起行列をコーパスから作るメソッドを実装する。

In [3]:
import numpy as np
import sys
sys.path.append("../../deep-learning-from-scratch-2")
from common.util import preprocess

In [4]:
def create_co_matrix(corpus, vocab_size, window_size = 1):
    corpus_size = len(corpus)
    co_matrix = np.zeros((vocab_size, vocab_size), dtype=np.int32)
    
    for idx, word_id in enumerate(corpus):
        for i in range(1, window_size + 1):
            left_idx = idx - i
            right_idx = idx + i
            
            if left_idx >= 0:
                left_word_id = corpus[left_idx]
                co_matrix[word_id, left_word_id] += 1
            
            if right_idx < corpus_size:
                right_word_id = corpus[right_idx]
                co_matrix[word_id, right_word_id] += 1

    return co_matrix
                

## 類似度
上記のメソッドとテキストをコーパスに変換するメソッドを使えば、テキストデータから単語を抜き出し、それぞれを整数値のベクトルに変換することができる。

では、単語同士が似ているかどうかはどうやって測ればいいだろう。そのために、コサイン類似度を定義する・

$$
similarity(x, y) = \frac{(x, y)}{\|x\| \|y\|} = ( \frac{x}{\|x\|} , \frac{y}{\|y\|} )
$$

つまり、ベクトルの内積をそれぞれのノルムで割るのである。もちろん、２次元であれば、２つのベクトルの間の角度に対するcosが出てくる。

内積、ノルムはL2で計算する。


In [7]:
# ベクトルを正規化してから内積を取る
# ゼロ割を防ぐために、小さな値(eps)を加算しておく
def cos_similarity(x, y, eps=1e-8):
    nx = x / (np.sqrt(np.sum(x**2)) + eps)
    ny = y / (np.sqrt(np.sum(y**2)) + eps)
    return np.dot(nx, ny)

In [8]:
# テキストデータから単語間の類似度を計算する
text = 'You say goodbye and i say hello.'
corpus ,word_to_id, id_to_word = preprocess(text)
vocab_size = len(word_to_id)
C = create_co_matrix(corpus, vocab_size)

c0 = C[word_to_id['you']]
c1 = C[word_to_id['i']]
cos_similarity(c0, c1)

0.70710676911547987

コサイン類似度は-1～1の値をとるので、youとiは比較的類似度が高いと言える。

## ランキング表示
類似度が計算できるようになったので、テキスト内の単語の中から、自分と似た単語をランキング形式で表示できるメソッドを実装する

In [11]:
# query: 検索対象の単語
# word_matrix:共起行列
def most_similar(query, word_to_id, id_to_word, word_matrix, top=5):
    if query not in word_to_id:
        print("not found")
        return
    
    print('\n[query] '+ query)
    query_id = word_to_id[query]
    query_vec = word_matrix[query_id]
    
    vocab_size = len(id_to_word)
    similarity = np.zeros(vocab_size)
    for i in range(vocab_size):
        similarity[i] = cos_similarity(word_matrix[i], query_vec)
    
    count = 0
    for i in (-1 * similarity).argsort():
        if id_to_word[i] == query:
            continue
        print('%s:  %s' % (id_to_word[i], similarity[i]))
        
        count += 1
        if count >= top:
            return

In [12]:
most_similar('you', word_to_id, id_to_word, C, top=5)


[query] you
goodbye:  0.707106769115
i:  0.707106769115
hello:  0.707106769115
say:  0.0
and:  0.0
